# Mosaic test
This notebook is used to test out cutting the larger complete map (made of large 1km * 1km mosaics) into smaller 500 * 500 pixel images.

- 1 tile = 1km * 1km
- 1 tile = 100.000cm * 100.000cm
- 100.000cm * 100.000cm / 8 cm per pixel = 12500 px * 12500 px
- (12500*12500)/(500*500) = 625
- final = 625 tiles of 500 8 500 pixels

# Imports

In [ ]:
import numpy as np
import rasterio
from rasterio.windows import Window

import xarray as xr
import xrspatial as xrs
import rioxarray as rxr

import stackstac as stc

tile_dir = "nl_8cm"
first_tile_dir = tile_dir + "/2022_128000_453000_RGB_hrl.tif"

# Reshaping
## Example of reshaping

In [ ]:
fake_tile = np.ones((12500, 12500))

In [ ]:
# fake_tile.reshape((-1,500,500)).shape
fake_tile.reshape((-1,500,500)).shape

## Reading first tile

In [ ]:
first_tile = rxr.open_rasterio(first_tile_dir)
first_tile = first_tile.rio.reproject("EPSG:28992")

In [ ]:
first_tile

In [ ]:
first_tile.plot.imshow()

In [ ]:
def mosaic_tile(tile_path, mosaic_size = 500):
    # Open the larger aerial image
    tile_name = tile_path.split("/")[-1].split(".tif")[0]
    with rasterio.open()  as src:
        # Get dimensions of the larger image
        height = src.height
        width = src.width
    
        # Calculate number of smaller mosaics along x and y axes
        num_tiles_x = width // mosaic_size
        num_tiles_y = height // mosaic_size
    
        # Iterate over each smaller mosaic
        for y in range(num_tiles_y):
            for x in range(num_tiles_x):
                # Define the window to read from
                window = Window(x * mosaic_size, y * mosaic_size, mosaic_size, mosaic_size)
    
                # Read the data from the larger image
                tile_data = src.read(window=window)
    
                # Define metadata for the smaller mosaic
                meta = src.meta.copy()
                meta['width'], meta['height'] = mosaic_size, mosaic_size
                meta['transform'] = rasterio.windows.transform(window, src.transform)
    
                # Write the smaller mosaic to disk
                with rasterio.open(f'mosaics/{tile_name}_{y}_{x}.tif', 'w', **meta) as dst:
                    dst.write(tile_data)

# Reading all tiles & splitting


In [ ]:
from glob import glob

In [ ]:
data_array = []

all_tiles = glob(tile_dir + "/2022*")
all_tiles[:10]

In [ ]:
[path.split("/")[-1].split(".tif")[0] for path in all_tiles]

In [ ]:
for tile_path in alltiles:
    mosaic_tile(tile_path)